<img src="../images/felipepelaquim-9c7CK4BemmA-unsplash.jpg"  style="float: center; margin: 0px; height: 350px; width: 1000px"> 

# Afrobeats influence in Hip Hop

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import time

In [50]:
#Authentication
client_credentials_manager = SpotifyClientCredentials(client_id = '2101cd224f5948e19c4c782d76744ed3',
                                                      client_secret = '0f81ac4a6af148138227fe1ee70ee64d')

Create spotipy object using SpotifyClientCredentials

In [78]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Get song info

Create a function that takes in a playlist of songs and extracts the songs information and features such as artist name, genre, album name, track_uri, danceability, energy, loudness, instrumentalness etc

In [52]:
def get_track_info(playlist):
    
    #split the playlist_uri is at the end of the playlists url. I'll use the .split method to extract it
    uri = playlist.split("/")[-1].split("?")[0]
    
    #from the spotipy library, use the playlist_tracks() method to extract each track from the playlist uri
    #It comes in a JSON format
    tracks = sp.playlist_tracks(uri)
    
    #create an empty dictionary with the info we want to extract as columns
    info = {
    'track_uri':[],
    'track_name':[],
    'artist_name':[],
    'artist_info':[],
    'artist_uri':[],
    'artist_popularity':[],
    'artist_genre':[],
    'album':[],
    'track_pop':[],
    }
    
    features = {'danceability': [],
     'energy': [],
     'key': [],
     'loudness': [],
     'mode': [],
     'speechiness': [],
     'acousticness': [],
     'instrumentalness': [],
     'liveness': [],
     'valence': [],
     'tempo': [],
     'type': [],
     'id': [],
     'uri': [],
     'track_href': [],
     'analysis_url': [],
     'duration_ms': [],
     'time_signature': []
               }
    
    #using a for loop, get the the info for each song and put it into the empty dictionary
    for track in tracks["items"]:
        #URI
        info['track_uri'].append((track["track"]["uri"]).split(':')[2])

        #Track name
        info['track_name'].append(track["track"]["name"])

        #Main Artist
        info['artist_uri'].append((track["track"]["artists"][0]["uri"]).split(':')[2])
        info['artist_info'].append(sp.artist(track["track"]["artists"][0]["uri"]))

        #Name, popularity, genre
        info['artist_name'].append(track["track"]["artists"][0]["name"])
        info['artist_popularity'].append(sp.artist(track["track"]["artists"][0]["uri"])["popularity"])
        info['artist_genre'].append(sp.artist(track["track"]["artists"][0]["uri"])["genres"])

        #Album
        info['album'].append(track["track"]["album"]["name"])

        #Popularity of the track
        info['track_pop'].append(track["track"]["popularity"])
        
        
        
        #loop through the tracks to their features and assign it to the empty dictionary
        
        track_uri = track["track"]["uri"].split(':')[2] 
        
        for key,value in (sp.audio_features(track_uri)[0]).items():
            features[key].append(value)
        
        time.sleep(1)
    #Transform the features dictionary into a dataframe
    features_df = pd.DataFrame(features)
    
    #drop features we do not need
    features_df.drop(columns=['type','id','uri','track_href','analysis_url','duration_ms','time_signature'], inplace=True)   
    
    #Transform the info dictionary into a dataframe
    info_df = pd.DataFrame(info)
    
     
    
    return info_df.join(features_df)
        
        
        

#### Create dataframe of users playlist

In [131]:
def raw_data(user_playlist_url, genre):
    user_playlist_info = get_track_info(user_playlist_url)
    #add user genre
    user_playlist_info.loc[:,'genre'] = genre
    return user_playlist_info

#### Afrobeats playlist

In [132]:
afrobeats_playlist_url= "https://open.spotify.com/playlist/5ZCzd0nCLqiIX1jwQWfazW"

In [133]:
afrobeats_df = raw_data(afrobeats_playlist_url, 'afrobeats')

#### Jacks playlist

In [137]:
jacks_playlist_url = 'https://open.spotify.com/playlist/6UlskZAcTPzcGMnQaMnIVm?si=5cbb031d1fdd4064'

In [138]:
jacks_playlist_df = raw_data(jacks_playlist_url, 'jack')

#### Ankita playlist

In [143]:
ankita_playlist_url = 'https://open.spotify.com/playlist/6qzbkhrmxdFj5TtaXR0sfI?si=ElY40mMjQ7apOsHTGlQI7A'

In [144]:
ankita_playlist_df = raw_data(ankita_playlist_url, 'ankita')

#### Export dataframes as a csv

In [139]:
afrobeats_df.to_csv('../data/afrobeats.csv', index = False)

In [140]:
jacks_playlist_df.to_csv('../data/jack.csv', index = False)

In [145]:
ankita_playlist_df.to_csv('../data/ankita.csv', index = False)